In [1]:
import torch  
from torch.utils.data import Dataset, DataLoader  
import torchvision.transforms as transforms  
from torchvision.datasets import ImageFolder  
from PIL import Image  
import pathlib  
import cupy as cp 
from torchvision.datasets import ImageFolder
from torchvision import datasets

In [2]:
# Verificar la disponibilidad de la GPU  
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
print("GPU disponible:", torch.cuda.is_available())
print("Nombre de la GPU:", torch.cuda.get_device_name() if torch.cuda.is_available() else "N/A")


GPU disponible: True
Nombre de la GPU: NVIDIA GeForce RTX 4060


In [3]:
# Definir directorios
directorio_dataset = '../Datasets/CIFAR-10 dos clases'
imagenes_originales = '../Datasets/CIFAR-10 dos clases/cat'
imagenes_corruptas = '../Datasets/CIFAR-10 dos clases/cat_c_defocus_blur'

directorio_dataset = pathlib.Path(directorio_dataset)
imagenes_originales = pathlib.Path(imagenes_originales)
imagenes_corruptas = pathlib.Path(imagenes_corruptas)



# Dimensiones de imagen y tamaño de batch
img_height = 32
img_width = 32
#batch_size = len(list(data_dir.glob('*/*.png'))) # leer todas las imágenes al tiempo


# Transformaciones para preprocesamiento de datos
transform = transforms.Compose([
    transforms.Resize((img_height, img_width)),  # Ajustar tamaño a 32x32
    transforms.ToTensor(),  # Convertir a tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalizar
])

class CustomImageDataset(Dataset):
    def __init__(self, directory, transform=None):
        self.directory = pathlib.Path(directory)
        self.transform = transform
        self.images = list(self.directory.glob('*.png'))

    def __len__(self):  
        return len(self.images)  

    def __getitem__(self, idx):  
        img_path = self.images[idx]  
        image = Image.open(img_path).convert('RGB')  
        if self.transform:  
            image = self.transform(image)  
        return image


# Crear datasets personalizados
dataset_completo = datasets.ImageFolder(root=directorio_dataset, transform=transform)
dataset_originales = CustomImageDataset(imagenes_originales, transform=transform)
dataset_corruptas = CustomImageDataset(imagenes_corruptas, transform=transform)

# Calcular totales
total_dataset = len(dataset_originales) + len(dataset_corruptas)
total_originales = len(dataset_originales)
total_corruptas = len(dataset_corruptas)

batch_size = total_dataset



# Crear el DataLoader para iterar sobre los datos en lotes
# DataLoader para cargar los datasets
dataloader_completo = DataLoader(dataset_completo, batch_size=batch_size, shuffle=False, num_workers=0)
dataloader_originales = DataLoader(dataset_originales, batch_size=batch_size, shuffle=False)
dataloader_corruptas = DataLoader(dataset_corruptas, batch_size=batch_size, shuffle=False)

# # DataLoader para cargar los datasets  
# dataloader_completo = DataLoader(dataset_completo, batch_size=batch_size, shuffle=False, num_workers=0)  

print(f"Total imágenes dataset: {total_dataset}")  
print(f"Total imágenes originales: {total_originales}")  
print(f"Total imágenes corruptas: {total_corruptas}")

Total imágenes dataset: 10000
Total imágenes originales: 5000
Total imágenes corruptas: 5000


In [4]:
img_height = 32
img_width = 32
batch_s = 32
channels = 3

In [5]:
def get_transform(channels):
    if channels == 3:
        # Definir transformaciones para el conjunto de datos en escala a color
        transform = transforms.Compose([
            transforms.Resize((img_height, img_width)),  # Ajustar tamaño
            transforms.ToTensor(),  # Convertir a tensor
            transforms.Normalize((0.5,), (0.5,))  # Normalizar
        ])
    elif channels == 1:
        # Definir transformaciones para el conjunto de datos en escala de grises
        transform = transforms.Compose([
            transforms.Grayscale(),  # Convertir a escala de grises
            transforms.Resize((img_height, img_width)),  # Ajustar tamaño
            transforms.ToTensor(),  # Convertir a tensor
            transforms.Normalize((0.5,), (0.5,))  # Normalizar
        ])
    else: 
        print("El valor elegido no está dentro del rango, por favor elija nuevamente")
        return None  # Devolver None en caso de elección inválida
    return transform

In [ ]:
# Obtener las transformaciones según la elección
transform = get_transform(channels)

# Crear datasets de entrenamiento y prueba desde el directorio
if transform is not None:
    train_dataset = datasets.ImageFolder(data_dir_train, transform=transform)
    test_dataset = datasets.ImageFolder(data_dir_test, transform=transform)
else:
    print("No se pueden crear los datasets debido a una elección inválida.")